In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf
import time

# to make this notebook's output stable across runs
def reset_graph(seed=42): 
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [2]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

# HW3.1

In [11]:
learning_rate = 0.001
batch_size = 16
train_size = X_train2.shape[0] # number of sample of training data
reset_graph()
with tf.Session() as sess:
    new_saver1 = tf.train.import_meta_graph('model/Team48_HW2.ckpt.meta', clear_devices=True) # import graph from HW2
    new_saver1.restore(sess, 'model/Team48_HW2.ckpt')
    
    # get Tensor from HW2 by name, the name was set in HW2
    xs = tf.get_default_graph().get_tensor_by_name("xs:0") # sample input 
    ys = tf.get_default_graph().get_tensor_by_name("ys:0") # label input
    y = tf.get_default_graph().get_tensor_by_name("y:0") # prediction output
    accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0") # accuracy rate 
    loss = tf.get_default_graph().get_tensor_by_name("loss:0") 
    
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits") # Variable of softmax layer
    optimizer = tf.train.AdamOptimizer(learning_rate, name="optimizer") # Set optimizer as Adam optimizer
    training_op = optimizer.minimize(loss, var_list=output_layer_vars, name="training_op") # Only train softmax layer
    
    tf.global_variables_initializer().run() 
    
    best_loss = 99999 # Initialize the best_loss as a large number, best_loss store the smallest validation loss
    stand_epoch = 0 # Initialize no progress times
    
    t0 = time.time() # Record the time before training loop
    
    # Strat training
    for i in range(1000):
        # Batch training, divide it to several batches
        for j in range(int(train_size/batch_size)): # number of batches: train_size/batch_size
            # get batch data each time
            batch_x = X_train2[j*batch_size:(j+1)*batch_size,:]
            batch_y = y_train2[j*batch_size:(j+1)*batch_size]
            # training
            sess.run(training_op, feed_dict={xs: batch_x, ys: batch_y})
        
        val_loss = sess.run(loss, feed_dict={xs: X_valid2, ys: y_valid2})  # loss for validation data
        
        accu = sess.run(accuracy, feed_dict={xs: X_test2, ys: y_test2}) # accuracy rate for testing data
        print(i,"Validation loss:", val_loss, " ; Best loss:", best_loss, " ; Accuracy:", accu)
        
        ### Check Early stop condition
        stand_epoch = stand_epoch + 1 # add 1 for each training epoch if validation loss has no progress
        if best_loss > val_loss:
            stand_epoch = 0 # If validation loss improve, reset no progress times to 0
            best_loss = val_loss 
        if stand_epoch >= 20: # If no progress times is over 20 epoches, early stop!!!
            print("Early stop in epoch", i)
            break

    t1 = time.time() # Record the time after training loop
    save_path = new_saver1.save(sess, "./model/Team48_HW3_1.ckpt") # save variables to checkpoint

sess.close()
print("Training time:", round(t1-t0,2), "sec") # Print out training time
print("Accuracy rate:", round(accu*100,3), "%")

INFO:tensorflow:Restoring parameters from model/Team48_HW2.ckpt
0 Validation loss: 232.55  ; Best loss: 99999  ; Accuracy: 0.333882
1 Validation loss: 225.232  ; Best loss: 232.55  ; Accuracy: 0.422547
2 Validation loss: 218.671  ; Best loss: 225.232  ; Accuracy: 0.50504
3 Validation loss: 212.781  ; Best loss: 218.671  ; Accuracy: 0.570253
4 Validation loss: 207.636  ; Best loss: 212.781  ; Accuracy: 0.606254
5 Validation loss: 203.254  ; Best loss: 207.636  ; Accuracy: 0.63238
6 Validation loss: 199.473  ; Best loss: 203.254  ; Accuracy: 0.653569
7 Validation loss: 196.166  ; Best loss: 199.473  ; Accuracy: 0.668175
8 Validation loss: 193.308  ; Best loss: 196.166  ; Accuracy: 0.679696
9 Validation loss: 190.892  ; Best loss: 193.308  ; Accuracy: 0.691216
10 Validation loss: 188.864  ; Best loss: 190.892  ; Accuracy: 0.696976
11 Validation loss: 187.146  ; Best loss: 188.864  ; Accuracy: 0.704587
12 Validation loss: 185.672  ; Best loss: 187.146  ; Accuracy: 0.710965
13 Validation lo

116 Validation loss: 165.792  ; Best loss: 165.826  ; Accuracy: 0.812178
117 Validation loss: 165.758  ; Best loss: 165.792  ; Accuracy: 0.812796
118 Validation loss: 165.725  ; Best loss: 165.758  ; Accuracy: 0.813001
119 Validation loss: 165.691  ; Best loss: 165.725  ; Accuracy: 0.813619
120 Validation loss: 165.658  ; Best loss: 165.691  ; Accuracy: 0.81403
121 Validation loss: 165.624  ; Best loss: 165.658  ; Accuracy: 0.813824
122 Validation loss: 165.59  ; Best loss: 165.624  ; Accuracy: 0.814853
123 Validation loss: 165.555  ; Best loss: 165.59  ; Accuracy: 0.814853
124 Validation loss: 165.52  ; Best loss: 165.555  ; Accuracy: 0.815264
125 Validation loss: 165.485  ; Best loss: 165.52  ; Accuracy: 0.815676
126 Validation loss: 165.449  ; Best loss: 165.485  ; Accuracy: 0.816499
127 Validation loss: 165.413  ; Best loss: 165.449  ; Accuracy: 0.815676
128 Validation loss: 165.377  ; Best loss: 165.413  ; Accuracy: 0.815881
129 Validation loss: 165.34  ; Best loss: 165.377  ; Acc

233 Validation loss: 162.258  ; Best loss: 162.273  ; Accuracy: 0.826373
234 Validation loss: 162.243  ; Best loss: 162.258  ; Accuracy: 0.826579
235 Validation loss: 162.228  ; Best loss: 162.243  ; Accuracy: 0.826579
236 Validation loss: 162.214  ; Best loss: 162.228  ; Accuracy: 0.826579
237 Validation loss: 162.201  ; Best loss: 162.214  ; Accuracy: 0.826579
238 Validation loss: 162.187  ; Best loss: 162.201  ; Accuracy: 0.826579
239 Validation loss: 162.175  ; Best loss: 162.187  ; Accuracy: 0.827402
240 Validation loss: 162.162  ; Best loss: 162.175  ; Accuracy: 0.827607
241 Validation loss: 162.15  ; Best loss: 162.162  ; Accuracy: 0.828225
242 Validation loss: 162.139  ; Best loss: 162.15  ; Accuracy: 0.828225
243 Validation loss: 162.128  ; Best loss: 162.139  ; Accuracy: 0.828225
244 Validation loss: 162.117  ; Best loss: 162.128  ; Accuracy: 0.828636
245 Validation loss: 162.107  ; Best loss: 162.117  ; Accuracy: 0.829047
246 Validation loss: 162.098  ; Best loss: 162.107  ;

# HW3.2

In [12]:
tf.reset_default_graph() 
with tf.Session() as sess:
    new_saver2 = tf.train.import_meta_graph('model/Team48_HW2.ckpt.meta', clear_devices=True)  # import graph from HW2 checkpoint
    new_saver2.restore(sess, 'model/Team48_HW2.ckpt')
    
    ### get Tensor from HW2 by name, the name was set in HW2
    xs = tf.get_default_graph().get_tensor_by_name("xs:0") # sample input 
    ys = tf.get_default_graph().get_tensor_by_name("ys:0") # label input
    y = tf.get_default_graph().get_tensor_by_name("y:0") # prediction output
    accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0") # accuracy rate 
    loss = tf.get_default_graph().get_tensor_by_name("loss:0")
    layer5 = tf.get_default_graph().get_tensor_by_name("l5:0") # 5th layer output tensor
    
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")  # Variable of softmax layer
    optimizer = tf.train.AdamOptimizer(learning_rate, name="optimizer") # Set optimizer as Adam optimizer
    training_op = optimizer.minimize(loss, var_list=output_layer_vars, name="training_op") # Only train softmax layer
    
    tf.global_variables_initializer().run()
    X_train_val = np.concatenate((X_train2, X_valid2)) # Combine training data and validation data
    # Feed training and validation data set into 5th layer and save the parameter in "layer5_cache"
    layer5_cache = sess.run(layer5, feed_dict={xs: X_train_val}) 
    
    best_loss = 99999 # Initialize the best_loss as a large number, best_loss store the smallest validation loss
    stand_epoch = 0 # Initialize no progress times
    
    t2 = time.time() # Record the time before training loop
    
    ### Start training
    for i in range(1000):
        ### Batch training, divide data to several batches
        for j in range(int(train_size/batch_size)):  # number of batches: train_size/batch_size
            # get batch data each time, Feed the 5th layer parameter as input when training softmax layer
            batch_x = layer5_cache[j*batch_size:(j+1)*batch_size,:] 
            batch_y = y_train2[j*batch_size:(j+1)*batch_size]
            # training
            sess.run(training_op, feed_dict={layer5: batch_x, ys: batch_y})
        
        val_loss = sess.run(loss, feed_dict={layer5: layer5_cache[500:,:], ys: y_valid2})  # loss for validation data
            
        accu = sess.run(accuracy, feed_dict={xs: X_test2, ys: y_test2}) # accuracy rate for testing data
        print(i,"Validation loss:", val_loss, " ; Best loss:", best_loss, " ; Accuracy:", accu) # result for each epoch
        
        ### Check Early stop condition
        stand_epoch = stand_epoch + 1 # add 1 for each training epoch if validation loss has no progress
        if best_loss > val_loss:
            stand_epoch = 0 # If validation loss improve, reset no progress times to 0
            best_loss = val_loss 
        if stand_epoch >= 20: # If no progress times is over 20 epoches, early stop!!!
            print("Early stop in epoch", i)
            break
    
    t3 = time.time()
    save_path = new_saver2.save(sess, "./model/Team48_HW3_2.ckpt") # save variables to checkpoint

sess.close()
print("Training time:", round(t3-t2,2), "sec") # Print out training time
print("Accuracy rate:", round(accu*100,3), "%")

INFO:tensorflow:Restoring parameters from model/Team48_HW2.ckpt
0 Validation loss: 232.55  ; Best loss: 99999  ; Accuracy: 0.333882
1 Validation loss: 225.232  ; Best loss: 232.55  ; Accuracy: 0.422547
2 Validation loss: 218.671  ; Best loss: 225.232  ; Accuracy: 0.50504
3 Validation loss: 212.781  ; Best loss: 218.671  ; Accuracy: 0.570253
4 Validation loss: 207.636  ; Best loss: 212.781  ; Accuracy: 0.606254
5 Validation loss: 203.254  ; Best loss: 207.636  ; Accuracy: 0.63238
6 Validation loss: 199.473  ; Best loss: 203.254  ; Accuracy: 0.653569
7 Validation loss: 196.166  ; Best loss: 199.473  ; Accuracy: 0.668175
8 Validation loss: 193.308  ; Best loss: 196.166  ; Accuracy: 0.679696
9 Validation loss: 190.892  ; Best loss: 193.308  ; Accuracy: 0.691216
10 Validation loss: 188.864  ; Best loss: 190.892  ; Accuracy: 0.696976
11 Validation loss: 187.146  ; Best loss: 188.864  ; Accuracy: 0.704588
12 Validation loss: 185.672  ; Best loss: 187.146  ; Accuracy: 0.710965
13 Validation lo

115 Validation loss: 165.826  ; Best loss: 165.86  ; Accuracy: 0.811561
116 Validation loss: 165.792  ; Best loss: 165.826  ; Accuracy: 0.812178
117 Validation loss: 165.758  ; Best loss: 165.792  ; Accuracy: 0.812796
118 Validation loss: 165.725  ; Best loss: 165.758  ; Accuracy: 0.813001
119 Validation loss: 165.691  ; Best loss: 165.725  ; Accuracy: 0.813619
120 Validation loss: 165.658  ; Best loss: 165.691  ; Accuracy: 0.81403
121 Validation loss: 165.624  ; Best loss: 165.658  ; Accuracy: 0.813824
122 Validation loss: 165.59  ; Best loss: 165.624  ; Accuracy: 0.814853
123 Validation loss: 165.555  ; Best loss: 165.59  ; Accuracy: 0.814853
124 Validation loss: 165.52  ; Best loss: 165.555  ; Accuracy: 0.815264
125 Validation loss: 165.485  ; Best loss: 165.52  ; Accuracy: 0.815676
126 Validation loss: 165.449  ; Best loss: 165.485  ; Accuracy: 0.816499
127 Validation loss: 165.413  ; Best loss: 165.449  ; Accuracy: 0.815676
128 Validation loss: 165.377  ; Best loss: 165.413  ; Acc

230 Validation loss: 162.305  ; Best loss: 162.321  ; Accuracy: 0.826373
231 Validation loss: 162.289  ; Best loss: 162.305  ; Accuracy: 0.826373
232 Validation loss: 162.273  ; Best loss: 162.289  ; Accuracy: 0.826373
233 Validation loss: 162.258  ; Best loss: 162.273  ; Accuracy: 0.826373
234 Validation loss: 162.243  ; Best loss: 162.258  ; Accuracy: 0.826579
235 Validation loss: 162.228  ; Best loss: 162.243  ; Accuracy: 0.826579
236 Validation loss: 162.214  ; Best loss: 162.228  ; Accuracy: 0.826579
237 Validation loss: 162.201  ; Best loss: 162.214  ; Accuracy: 0.826579
238 Validation loss: 162.187  ; Best loss: 162.201  ; Accuracy: 0.826579
239 Validation loss: 162.175  ; Best loss: 162.187  ; Accuracy: 0.827402
240 Validation loss: 162.162  ; Best loss: 162.175  ; Accuracy: 0.827607
241 Validation loss: 162.15  ; Best loss: 162.162  ; Accuracy: 0.828225
242 Validation loss: 162.139  ; Best loss: 162.15  ; Accuracy: 0.828225
243 Validation loss: 162.128  ; Best loss: 162.139  ;

# HW3.3

In [13]:
tf.reset_default_graph() 
with tf.Session() as sess:
    new_saver3 = tf.train.import_meta_graph('model/Team48_HW2.ckpt.meta', clear_devices=True)  # import graph from HW2 checkpoint
    new_saver3.restore(sess, 'model/Team48_HW2.ckpt')
    
    ### get Tensor from HW2 by name, the name was set in HW2
    xs = tf.get_default_graph().get_tensor_by_name("xs:0") # sample input 
    ys = tf.get_default_graph().get_tensor_by_name("ys:0") # label input
    layer4 = tf.get_default_graph().get_tensor_by_name("l4:0") # 4th layer output tensor
    
    ### Create a new softmax layer
    Weights = tf.get_variable("new_softmax3", shape=[128, 5], initializer=tf.contrib.layers.variance_scaling_initializer())
    third_logits = tf.matmul(layer4, Weights) # Connect 4th layer to new softmax layer
    y = tf.nn.softmax(third_logits, name="third_y") # prediction of model
    cross_entropy = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = ys, logits = y), name="third_loss") # Cost
    optm = tf.train.AdamOptimizer(learning_rate, name="optm") # Adam optimizer
    train_step = optm.minimize(cross_entropy, var_list=Weights, name="training_step") # Only train softmax layer
    accuracy_rate = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), ys), tf.float32), name = "third_accuracy") # Calculate accuracy rate
    
    tf.global_variables_initializer().run() 
    
    best_loss = 99999 # Initialize the best_loss as a large number, best_loss store the smallest validation loss
    stand_epoch = 0 # Initialize no progress times
    
    t4 = time.time() # Record the time before training loop
    
    for i in range(1000):
        ### Batch training, divide data to several batches
        for j in range(int(train_size/batch_size)):  # number of batches: train_size/batch_size
            # get batch data each time
            batch_x = X_train2[j*batch_size:(j+1)*batch_size,:]
            batch_y = y_train2[j*batch_size:(j+1)*batch_size]
            # training
            sess.run(train_step, feed_dict={xs: batch_x, ys: batch_y})
        
        val_loss = sess.run(cross_entropy, feed_dict={xs: X_valid2, ys: y_valid2})  # loss for validation data            
        
        accu = sess.run(accuracy_rate, feed_dict={xs: X_test2, ys: y_test2}) # accuracy rate for testing data
        print(i,"Validation loss:", val_loss, " ; Best loss:", best_loss, " ; Accuracy:", accu) # result for each epoch
        
        ### Check Early stop condition
        stand_epoch = stand_epoch + 1 # add 1 for each training epoch if validation loss has no progress
        if best_loss > val_loss:
            stand_epoch = 0 # If validation loss improve, reset no progress times to 0
            best_loss = val_loss 
        if stand_epoch >= 20: # If no progress times is over 20 epoches, early stop!!!
            print("Early stop in epoch", i)
            break
    t5 = time.time() # Record the time after training loop
    save_path = new_saver3.save(sess, "./model/Team48_HW3_3.ckpt") # save variables to checkpoint

sess.close()
print("Training time:", round(t5-t4,2), "sec") # Print out training time
print("Accuracy rate:", round(accu*100,3), "%")

INFO:tensorflow:Restoring parameters from model/Team48_HW2.ckpt
0 Validation loss: 237.411  ; Best loss: 99999  ; Accuracy: 0.2662
1 Validation loss: 231.095  ; Best loss: 237.411  ; Accuracy: 0.363917
2 Validation loss: 224.336  ; Best loss: 231.095  ; Accuracy: 0.455256
3 Validation loss: 217.547  ; Best loss: 224.336  ; Accuracy: 0.538778
4 Validation loss: 211.339  ; Best loss: 217.547  ; Accuracy: 0.596379
5 Validation loss: 206.013  ; Best loss: 211.339  ; Accuracy: 0.63814
6 Validation loss: 201.564  ; Best loss: 206.013  ; Accuracy: 0.664884
7 Validation loss: 197.878  ; Best loss: 201.564  ; Accuracy: 0.680724
8 Validation loss: 194.804  ; Best loss: 197.878  ; Accuracy: 0.694301
9 Validation loss: 192.203  ; Best loss: 194.804  ; Accuracy: 0.706233
10 Validation loss: 189.968  ; Best loss: 192.203  ; Accuracy: 0.715902
11 Validation loss: 188.025  ; Best loss: 189.968  ; Accuracy: 0.723925
12 Validation loss: 186.319  ; Best loss: 188.025  ; Accuracy: 0.733182
13 Validation l

115 Validation loss: 164.849  ; Best loss: 164.876  ; Accuracy: 0.826167
116 Validation loss: 164.822  ; Best loss: 164.849  ; Accuracy: 0.825962
117 Validation loss: 164.794  ; Best loss: 164.822  ; Accuracy: 0.826167
118 Validation loss: 164.767  ; Best loss: 164.794  ; Accuracy: 0.825756
119 Validation loss: 164.74  ; Best loss: 164.767  ; Accuracy: 0.825756
120 Validation loss: 164.712  ; Best loss: 164.74  ; Accuracy: 0.826373
121 Validation loss: 164.685  ; Best loss: 164.712  ; Accuracy: 0.826784
122 Validation loss: 164.657  ; Best loss: 164.685  ; Accuracy: 0.826373
123 Validation loss: 164.629  ; Best loss: 164.657  ; Accuracy: 0.82699
124 Validation loss: 164.601  ; Best loss: 164.629  ; Accuracy: 0.82699
125 Validation loss: 164.573  ; Best loss: 164.601  ; Accuracy: 0.827196
126 Validation loss: 164.545  ; Best loss: 164.573  ; Accuracy: 0.828019
127 Validation loss: 164.516  ; Best loss: 164.545  ; Accuracy: 0.82843
128 Validation loss: 164.488  ; Best loss: 164.516  ; Ac

231 Validation loss: 162.118  ; Best loss: 162.131  ; Accuracy: 0.837482
232 Validation loss: 162.105  ; Best loss: 162.118  ; Accuracy: 0.837482
233 Validation loss: 162.092  ; Best loss: 162.105  ; Accuracy: 0.837688
234 Validation loss: 162.08  ; Best loss: 162.092  ; Accuracy: 0.837893
235 Validation loss: 162.068  ; Best loss: 162.08  ; Accuracy: 0.837893
236 Validation loss: 162.056  ; Best loss: 162.068  ; Accuracy: 0.837688
237 Validation loss: 162.045  ; Best loss: 162.056  ; Accuracy: 0.837482
238 Validation loss: 162.034  ; Best loss: 162.045  ; Accuracy: 0.837482
239 Validation loss: 162.023  ; Best loss: 162.034  ; Accuracy: 0.837688
240 Validation loss: 162.012  ; Best loss: 162.023  ; Accuracy: 0.837482
241 Validation loss: 162.002  ; Best loss: 162.012  ; Accuracy: 0.837688
242 Validation loss: 161.993  ; Best loss: 162.002  ; Accuracy: 0.837688
243 Validation loss: 161.983  ; Best loss: 161.993  ; Accuracy: 0.837688
244 Validation loss: 161.974  ; Best loss: 161.983  ;

# HW 3.4

In [14]:
tf.reset_default_graph() 
with tf.Session() as sess:
    new_saver4 = tf.train.import_meta_graph('model/Team48_HW2.ckpt.meta', clear_devices=True)  # import graph from HW2 checkpoint
    new_saver4.restore(sess, 'model/Team48_HW2.ckpt')
    
    ### get Tensor from HW2 by name, the name was set in HW2
    xs = tf.get_default_graph().get_tensor_by_name("xs:0") # sample input 
    ys = tf.get_default_graph().get_tensor_by_name("ys:0") # label input
    layer4 = tf.get_default_graph().get_tensor_by_name("l4:0") # 4th layer output tensor
    
    layer1_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="l1") # get Variable of 1st layer
    layer2_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="l2") # get Variable of 2nd layer
    
    ### Create a new softmax layer
    Weights = tf.get_variable("new_softmax4", shape=[128, 5], initializer=tf.contrib.layers.variance_scaling_initializer())
    third_logits = tf.matmul(layer4, Weights) # Connect 4th layer to new softmax layer
    y = tf.nn.softmax(third_logits, name="third_y") # prediction value of model
    cross_entropy = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = ys, logits = y), name="third_loss") # Cost
    optm = tf.train.AdamOptimizer(learning_rate, name="optm") # Adam optimizer
    train_step = optm.minimize(cross_entropy, var_list=[layer1_vars, layer1_vars, Weights], name="training_step") # Optimizer
    accuracy_rate = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), ys), tf.float32), name = "third_accuracy") # Calculate accuracy rate
    
    tf.global_variables_initializer().run() 
    
    best_loss = 99999 # Initialize the best_loss as a large number, best_loss store the smallest validation loss
    stand_epoch = 0 # Initialize no progress times
    
    t6 = time.time() # Record the time before training loop
    
    for i in range(1000):
        ### Batch training, divide data to several batches
        for j in range(int(train_size/batch_size)): # number of batches: train_size/batch_size
            # get batch data each time
            batch_x = X_train2[j*batch_size:(j+1)*batch_size,:]
            batch_y = y_train2[j*batch_size:(j+1)*batch_size]
            # training
            sess.run(train_step, feed_dict={xs: batch_x, ys: batch_y})
        
        val_loss = sess.run(cross_entropy, feed_dict={xs: X_valid2, ys: y_valid2}) # loss for validation data            
        accu = sess.run(accuracy_rate, feed_dict={xs: X_test2, ys: y_test2}) # accuracy rate for testing data
        print(i,"Validation loss:", val_loss, " ; Best loss:", best_loss, " ; Accuracy:", accu) # result for each epoch
        
        ### Check Early stop condition
        stand_epoch = stand_epoch + 1 # add 1 for each training epoch if validation loss has no progress
        if best_loss > val_loss:
            stand_epoch = 0 # If validation loss improve, reset no progress times to 0
            best_loss = val_loss 
        if stand_epoch >= 20: # If no progress times is over 20 epoches, early stop!!!
            print("Early stop in epoch", i)
            break
    t7 = time.time() # Record the time after training loop
    save_path = new_saver4.save(sess, "./model/Team48_HW3_4.ckpt") # save variables to checkpoint

sess.close()
print("Training time:", round(t7-t6,2), "sec") # Print out training time
print("Accuracy rate:", round(accu*100,3), "%")

INFO:tensorflow:Restoring parameters from model/Team48_HW2.ckpt
0 Validation loss: 255.722  ; Best loss: 99999  ; Accuracy: 0.183501
1 Validation loss: 255.712  ; Best loss: 255.722  ; Accuracy: 0.183707
2 Validation loss: 239.551  ; Best loss: 255.712  ; Accuracy: 0.289652
3 Validation loss: 231.127  ; Best loss: 239.551  ; Accuracy: 0.237605
4 Validation loss: 204.785  ; Best loss: 231.127  ; Accuracy: 0.498457
5 Validation loss: 204.275  ; Best loss: 204.785  ; Accuracy: 0.531166
6 Validation loss: 203.723  ; Best loss: 204.275  ; Accuracy: 0.535487
7 Validation loss: 203.445  ; Best loss: 203.723  ; Accuracy: 0.541247
8 Validation loss: 178.545  ; Best loss: 203.445  ; Accuracy: 0.685044
9 Validation loss: 186.34  ; Best loss: 178.545  ; Accuracy: 0.597202
10 Validation loss: 178.2  ; Best loss: 178.545  ; Accuracy: 0.681958
11 Validation loss: 168.956  ; Best loss: 178.2  ; Accuracy: 0.771035
12 Validation loss: 180.797  ; Best loss: 168.956  ; Accuracy: 0.69389
13 Validation loss

114 Validation loss: 146.562  ; Best loss: 146.044  ; Accuracy: 0.917712
115 Validation loss: 146.576  ; Best loss: 146.044  ; Accuracy: 0.917712
116 Validation loss: 146.59  ; Best loss: 146.044  ; Accuracy: 0.917918
117 Validation loss: 146.602  ; Best loss: 146.044  ; Accuracy: 0.917918
118 Validation loss: 146.615  ; Best loss: 146.044  ; Accuracy: 0.917918
119 Validation loss: 146.627  ; Best loss: 146.044  ; Accuracy: 0.917918
120 Validation loss: 146.64  ; Best loss: 146.044  ; Accuracy: 0.917918
121 Validation loss: 146.652  ; Best loss: 146.044  ; Accuracy: 0.917918
122 Validation loss: 146.663  ; Best loss: 146.044  ; Accuracy: 0.917918
123 Validation loss: 146.676  ; Best loss: 146.044  ; Accuracy: 0.918124
Early stop in epoch 123
Training time: 12.02 sec
Accuracy rate: 91.812 %
